In [77]:
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader
import torch.utils.data as data

train_x = [torch.Tensor([[1, 1], [1, 1], [1, 1], [1,1]]),
           torch.Tensor([[2, 2], [2, 2], [2, 2]]),
           torch.Tensor([[3, 3], [3, 3]]),
           torch.Tensor([[4, 4], [4, 4]]),
           torch.Tensor([[5, 5]]),
           torch.Tensor([[6,6]])
        ]
print(train_x)

[tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]]), tensor([[2., 2.],
        [2., 2.],
        [2., 2.]]), tensor([[3., 3.],
        [3., 3.]]), tensor([[4., 4.],
        [4., 4.]]), tensor([[5., 5.]]), tensor([[6., 6.]])]


In [78]:
class MyData(data.Dataset):
    def __init__(self, data_seq):
        self.data_seq = data_seq

    def __len__(self):
        return len(self.data_seq)

    def __getitem__(self, idx):
        return self.data_seq[idx]
    
def collate_fn(data):
    data.sort(key=lambda x: len(x), reverse=True)
    data_length = [len(sq) for sq in data]
    data = rnn_utils.pad_sequence(data, batch_first=True, padding_value=0)
    return data, data_length

batch_size=2
if __name__=='__main__':
    data = MyData(train_x)
    data_loader = DataLoader(data, batch_size=batch_size, shuffle=True, 
                             collate_fn=collate_fn)
    batch_x, batch_x_len = iter(data_loader).next()
    batch_x_pack = rnn_utils.pack_padded_sequence(batch_x, 
                                                  batch_x_len, batch_first=True)
print(batch_x,batch_x_len)
print(batch_x_pack)

tensor([[[2., 2.],
         [2., 2.],
         [2., 2.]],

        [[4., 4.],
         [4., 4.],
         [0., 0.]]]) [3, 2]
PackedSequence(data=tensor([[2., 2.],
        [4., 4.],
        [2., 2.],
        [4., 4.],
        [2., 2.]]), batch_sizes=tensor([2, 2, 1]))


In [191]:
def pack(data):
    data = MyData(data)
    data_loader = DataLoader(data, batch_size=batch_size, shuffle=True, 
                             collate_fn=collate_fn)
    batch_x, batch_x_len = iter(data_loader).next()
    batch_x_pack = rnn_utils.pack_padded_sequence(batch_x, 
                                                  batch_x_len, batch_first=True)
    return batch_x_pack

In [229]:
#RNN
num_layers=1
input_size=2
output_size=2
output_size_h=2
net = nn.RNN(input_size, output_size, num_layers)#输入维度2，输出维度3
h0 = torch.rand(num_layers, batch_size, output_size)
#print(batch_x_pack.data.shape)#
out, (h1) = net(batch_x_pack,h0)
#print(out.data.shape)#
print(h1)#

tensor([[[ 0.5153, -0.5102],
         [ 0.9747,  0.0218]]], grad_fn=<StackBackward>)


In [230]:
#Linear
Linear=nn.Linear(input_size,output_size)
output=Linear(batch_x_pack.data)
print(output)

tensor([[ 2.0356, -0.9986],
        [ 3.4362, -1.4593],
        [ 2.0356, -0.9986],
        [ 3.4362, -1.4593],
        [ 2.0356, -0.9986]], grad_fn=<AddmmBackward>)


In [231]:
h0 = torch.rand(1, 2, 2)#batch_first=False
net(output.data[0:2].reshape(1,2,-1),h0)

(tensor([[[-0.8476, -0.9926],
          [-0.8462, -0.9984]]], grad_fn=<StackBackward>),
 tensor([[[-0.8476, -0.9926],
          [-0.8462, -0.9984]]], grad_fn=<StackBackward>))

In [232]:
#X与h的shape对齐。
h_size=h0.shape[1]
xdata=output.data[0:2].reshape(1,2,-1)
print(h_size)
x_size=xdata.shape[1]
print(x_size)
if (h_size<x_size):
    xdata=xdata[:,0:h_size,:]
print(xdata)

2
2
tensor([[[ 2.0356, -0.9986],
         [ 3.4362, -1.4593]]])


In [236]:
output_size_o=2
net_o=nn.RNN(output_size_h, output_size_o, num_layers)#输入维度2，输出维度3

In [247]:
#for循环获取h的batch
#X to O:
length=0
xdatas=[]
h_=[]
outs_o=[]
#
h_size=batch_x_pack.batch_sizes[0]
h0= batch_x_pack.data[0:h_size].reshape(1,h_size,-1)
h=torch.zeros(h0.shape)
print('h0')
print(h0)
print(h)
out_o,_=net_o(h0,h)
outs_o.append(out_o)
print(out_o)
for index in range(len(batch_x_pack.batch_sizes)-1):
    h_batch_size=batch_x_pack.batch_sizes[index]
    print(h_batch_size)
    if(len(xdatas)==2):
        del xdatas[0]
    out=output.data[length:length+h_batch_size].reshape(1,h_batch_size,-1)
    length=length+h_batch_size
    h_size=batch_x_pack.batch_sizes[index+1]
    h0= batch_x_pack.data[length:length+h_size].reshape(1,h_size,-1)
    print('h0')
    print(h0)
    
    #size 调整
    print(h_size)
    xdatas.append(out[:,0:h_size,:])
    xdatas[0]=xdatas[0][:,0:h_size,:]
    print(xdatas)
    
    #pack
    train_h=xdatas
    print('train_h')
    print(train_h)
    if(len(xdatas)==2):
        train_h=torch.stack((xdatas[0],xdatas[1]),dim=1).reshape(2,h_size,-1)
    else:
        train_h=xdatas[0]
    print(train_h)
    #Net
    out,h=net(train_h,h0)
    h_.append(h)
    print('out')
    print(out)
    print('h')
    print(h)
    out_o,_=net_o(h0,h)
    outs_o.append(out_o)
    print('out_o')
    print(out_o)

h0
tensor([[[2., 2.],
         [4., 4.]]])
tensor([[[0., 0.],
         [0., 0.]]])
tensor([[[ 0.7687, -0.7467],
         [ 0.9785, -0.9673]]], grad_fn=<StackBackward>)
tensor(2)
h0
tensor([[[2., 2.],
         [4., 4.]]])
tensor(2)
[tensor([[[ 2.0356, -0.9986],
         [ 3.4362, -1.4593]]])]
train_h
[tensor([[[ 2.0356, -0.9986],
         [ 3.4362, -1.4593]]])]
tensor([[[ 2.0356, -0.9986],
         [ 3.4362, -1.4593]]])
out
tensor([[[-0.9597, -0.9508],
         [-0.9891, -0.9529]]], grad_fn=<StackBackward>)
h
tensor([[[-0.9597, -0.9508],
         [-0.9891, -0.9529]]], grad_fn=<StackBackward>)
out_o
tensor([[[ 0.8790, -0.1518],
         [ 0.9892, -0.8416]]], grad_fn=<StackBackward>)
tensor(2)
h0
tensor([[[2., 2.]]])
tensor(1)
[tensor([[[ 2.0356, -0.9986]]]), tensor([[[ 2.0356, -0.9986]]])]
train_h
[tensor([[[ 2.0356, -0.9986]]]), tensor([[[ 2.0356, -0.9986]]])]
tensor([[[ 2.0356, -0.9986]],

        [[ 2.0356, -0.9986]]])
out
tensor([[[-0.9597, -0.9508]],

        [[-0.6509, -0.9976]]], 

In [248]:
#x2开始
print(outs_o)#一个完整的batch的输出O，o1,o2,o3...

[tensor([[[ 0.7687, -0.7467],
         [ 0.9785, -0.9673]]], grad_fn=<StackBackward>), tensor([[[ 0.8790, -0.1518],
         [ 0.9892, -0.8416]]], grad_fn=<StackBackward>), tensor([[[ 0.8988, -0.2122]]], grad_fn=<StackBackward>)]


In [ ]:
#O to Y:
